In [11]:
import streamlit as st
import pandas as pd
import numpy as np
import folium
from streamlit_folium import folium_static
import json
from pandas import json_normalize
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
# Liens de téléchargement des données
# data20XX : allocataires APA puy de dome de 20XX
data2019_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1460_2020-09-03.csv")

data2020_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1492_2021-03-18.csv")

data2018_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1458_2020-09-03.csv")

data2017_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1456_2020-09-03.csv")

data2016_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1454_2020-09-03.csv")

data2015_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1452_2020-09-03.csv")

data2014_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1450_2020-09-03.csv")

# datageo : coordonnées des communes du puy de dome
datageocommune_url = (
    'https://github.com/gregoiredavid/france-geojson/raw/master/departements/63-puy-de-dome/communes-63-puy-de-dome.geojson')
geocommune=pd.read_json(datageocommune_url)

datageodepartement_url = (
'https://github.com/gregoiredavid/france-geojson/raw/master/departements/63-puy-de-dome/departement-63-puy-de-dome.geojson')
geodepartement=pd.read_json(datageodepartement_url)

df2020 = pd.read_csv(data2020_url, sep=';', encoding = "ISO-8859-1")
df2019 = pd.read_csv(data2019_url, sep=';', encoding = "ISO-8859-1")
df2018 = pd.read_csv(data2018_url, sep=';', encoding = "ISO-8859-1")
df2017 = pd.read_csv(data2017_url, sep=';', encoding = "ISO-8859-1")
df2016 = pd.read_csv(data2016_url, sep=';', encoding = "ISO-8859-1")
df2015 = pd.read_csv(data2015_url, sep=';', encoding = "ISO-8859-1")
df2014 = pd.read_csv(data2014_url, sep=';', encoding = "ISO-8859-1")

In [13]:
# Notebook de nettoyage des données

col_suppr = ['DATE_DEBUT', 'DATE_FIN', 'ID_HISTORIQUE']
col_texte = ['CODE_COMMUNE', 'CODE_CANTON']
col_nombre = ['TOTAL', 'TOTAL_FEM', 'TOTAL_HOM', 'TOTAL_M75A', 'TOTAL_7584A', 'TOTAL_P85A', 'TOTAL_DOM', 'TOTAL_ETA']

def nettoie(A):
    A.columns = A.loc[0]
    A.drop(0,axis=0,inplace=True)
    A.drop(col_suppr, axis=1, inplace=True)
    A.fillna(0, inplace=True)
    A[col_texte] = A[col_texte].astype('str')
    A[col_nombre] = A[col_nombre].astype('int')
    return A

nettoie(df2020)
nettoie(df2019)
nettoie(df2018)
nettoie(df2017)
nettoie(df2016)
nettoie(df2015)
nettoie(df2014)

def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

dfgeocommune = json_normalize(geocommune['features'])
dfgeocommune.drop('type', axis=1, inplace=True)
dfgeocommune.drop('geometry.type', axis=1, inplace=True)
dfgeocommune=dfgeocommune.rename(columns={"geometry.coordinates": "coordinates",
                      "properties.code": "CODE_COMMUNE",
                      "properties.nom": "LIBL_COMMUNE"})
dfgeocommune['LIBL_COMMUNE']=dfgeocommune['LIBL_COMMUNE'].str.upper()

for commune in range(len(dfgeocommune['LIBL_COMMUNE'])):
    dfgeocommune['LIBL_COMMUNE'][commune] = remove_accents(dfgeocommune['LIBL_COMMUNE'][commune])

liste_coord=[]
for i in range(len(dfgeocommune)):
    point=[]
    if len(dfgeocommune['coordinates'][i])==1:
        for j in range(len(dfgeocommune['coordinates'][i][0])):
            truc=[]
            truc.append(dfgeocommune['coordinates'][i][0][j][1]) 
            truc.append(dfgeocommune['coordinates'][i][0][j][0])
            point.append(truc)
    else:
        for j in range(len(dfgeocommune['coordinates'][i])):
            truc=[]
            for k in range(len(dfgeocommune['coordinates'][i][j][0])):
                machin=[]
                machin.append(dfgeocommune['coordinates'][i][j][0][k][1])
                machin.append(dfgeocommune['coordinates'][i][j][0][k][0])
                truc.append(machin)
            point.append(truc)
    liste_coord.append(point)
dfgeocommune['geo']=liste_coord

dfgeocommune.drop(['coordinates'], axis=1, inplace=True)

pointd=[]
for j in range(len(geodepartement['geometry'][1][0])):
    truc=[]
    truc.append(geodepartement['geometry'][1][0][j][1])
    truc.append(geodepartement['geometry'][1][0][j][0])
    pointd.append(truc)
dfgeodepartement=pd.DataFrame(columns=['geo'], index=['points'])
dfgeodepartement['geo'][0]=pointd


In [14]:
dfgeocommune[(dfgeocommune['LIBL_COMMUNE']=='CRESTE') | (dfgeocommune['LIBL_COMMUNE']=='SAINT-DIERY')]

,CODE_COMMUNE,LIBL_COMMUNE,geo
76,63127,CRESTE,"[[45.54058, 3.03184], [45.54158, 3.02987], [45..."
206,63335,SAINT-DIERY,"[[45.51164, 3.00339], [45.51813, 3.0011], [45...."


In [15]:
# Calcul des coordonnées de la nouvelle commune de Saint Diery = fusion de Saint Diery et Creste en 2019

liste_st_diery=[]
liste_st_diery=dfgeocommune[dfgeocommune['LIBL_COMMUNE']=='SAINT-DIERY']['geo'].tolist()[0]

liste_creste=[]
liste_creste=dfgeocommune[dfgeocommune['LIBL_COMMUNE']=='CRESTE']['geo'].tolist()[0]

liste_commune=[]
for i in range(len(liste_st_diery)):
    if liste_st_diery[i] in liste_creste:
        liste_commune.append(liste_st_diery[i])

if liste_creste.index(liste_commune[0])-liste_creste.index(liste_commune[1])<0:
    liste_creste.reverse()

new1 = liste_st_diery[0:liste_st_diery.index(liste_commune[0])+1]
new2 = liste_creste[liste_creste.index(liste_commune[0])+1:len(liste_creste)-1]
new3 = liste_creste[0:liste_creste.index(liste_commune[-1])+1]
new4 = liste_st_diery[liste_st_diery.index(liste_commune[-1])+1:len(liste_st_diery)-1]

newliste_st_diery = new1 + new2 + new3 + new4
newliste_st_diery.append(newliste_st_diery[0])

dfgeocommune['geo'][206]=newliste_st_diery

In [16]:
dfgeocommune[(dfgeocommune['LIBL_COMMUNE']=='CHAMEANE') | (dfgeocommune['LIBL_COMMUNE']=='VERNET-LA-VARENNE')]

,CODE_COMMUNE,LIBL_COMMUNE,geo
99,63078,CHAMEANE,"[[45.53241, 3.40553], [45.53338, 3.40773], [45..."
144,63448,VERNET-LA-VARENNE,"[[45.47508, 3.40289], [45.4762, 3.40342], [45...."


In [17]:
# Calcul des coordonnées de la nouvelle commune de Le Vernet-Chaméane = fusion de Vernet la Varenne et Chaméane en 2019

liste_vernet=[]
liste_vernet=dfgeocommune[dfgeocommune['LIBL_COMMUNE']=='VERNET-LA-VARENNE']['geo'].tolist()[0]

liste_chameane=[]
liste_chameane=dfgeocommune[dfgeocommune['LIBL_COMMUNE']=='CHAMEANE']['geo'].tolist()[0]

liste_commune=[]
for i in range(len(liste_vernet)):
    if liste_vernet[i] in liste_chameane:
        liste_commune.append(liste_vernet[i])

if liste_chameane.index(liste_commune[0])-liste_chameane.index(liste_commune[1])<0:
    liste_chameane.reverse()

new1 = liste_vernet[0:liste_vernet.index(liste_commune[0])+1]
new2 = liste_chameane[liste_chameane.index(liste_commune[0])+1:len(liste_chameane)-1]
new3 = liste_chameane[0:liste_chameane.index(liste_commune[-1])+1]
new4 = liste_vernet[liste_vernet.index(liste_commune[-1])+1:len(liste_vernet)-1]

newliste_vernet = new1 + new2 + new3 + new4
newliste_vernet.append(newliste_vernet[0])

dfgeocommune['geo'][144]=newliste_vernet

In [18]:
dfgeocommune[(dfgeocommune['LIBL_COMMUNE']=='DALLET') | (dfgeocommune['LIBL_COMMUNE']=='MEZEL')]

,CODE_COMMUNE,LIBL_COMMUNE,geo
315,63133,DALLET,"[[45.78586, 3.25916], [45.78121, 3.26094], [45..."
407,63226,MEZEL,"[[45.73407, 3.21602], [45.73519, 3.21636], [45..."


In [19]:
# Calcul des coordonnées de la nouvelle commune de Mur sur Allier = fusion de Mezel et Dallet en 2019

liste_mezel=[]
liste_mezel=dfgeocommune[dfgeocommune['LIBL_COMMUNE']=='MEZEL']['geo'].tolist()[0]

liste_dallet=[]
liste_dallet=dfgeocommune[dfgeocommune['LIBL_COMMUNE']=='DALLET']['geo'].tolist()[0]

liste_commune=[]
for i in range(len(liste_mezel)):
    if liste_mezel[i] in liste_dallet:
        liste_commune.append(liste_mezel[i])

if liste_dallet.index(liste_commune[0])-liste_dallet.index(liste_commune[1])<0:
    liste_dallet.reverse()

new1 = liste_mezel[0:liste_mezel.index(liste_commune[0])+1]
new2 = liste_dallet[liste_dallet.index(liste_commune[0])+1:len(liste_dallet)-1]
new3 = liste_dallet[0:liste_dallet.index(liste_commune[-1])+1]
new4 = liste_mezel[liste_mezel.index(liste_commune[-1])+1:len(liste_mezel)-1]

newliste_mezel = new1 + new2 + new3 + new4
newliste_mezel.append(liste_mezel[0])

dfgeocommune['geo'][407]=newliste_mezel

In [20]:
#Export des fichiers
dfgeocommune.to_csv('commune_geo.csv')
dfgeodepartement.to_csv('departement_geo.csv')

df2020.to_csv('APA_2020.csv')
df2019.to_csv('APA_2019.csv')
df2018.to_csv('APA_2018.csv')
df2017.to_csv('APA_2017.csv')
df2016.to_csv('APA_2016.csv')
df2015.to_csv('APA_2015.csv')
df2014.to_csv('APA_2014.csv')